In [6]:
import pandas as pd
import json
import re

In [7]:
# csv_path = '../data/Respuestas_Whatss_MX.csv'
csv_path = '../data/raw/MEX_WA_Messages.txt'

In [8]:
# mx_df = pd.read_csv(csv_path, encoding='utf-8', sep=',')
mx_df = pd.read_csv(csv_path, sep='\t', lineterminator='\r')

In [9]:
mx_df.head()

,mobile,locale,createddate,message_hcp_reply,ConversationId,ChatMessagingMOLogId,ChannelId
0,524499287753,MX,Apr 1 2025 8:20PM,"{""Sha256"":""oXTLNxWj4wcX1sP12vU2CQ9pbfS1ZOnvbqW...",wamid.HBgNNTIxNDQ5OTI4Nzc1MxUCABIYIDM4MTc1MEI5...,50647,5215662260886
1,526641650361,MX,Apr 1 2025 10:13PM,"{""Sha256"":""I7M+qvtt/mpnxGvgwgS6UedqoqKl3X26r5/...",wamid.HBgNNTIxNjY0MTY1MDM2MRUCABIYFDNBNzFFNTkx...,50648,5215662260886
2,524777295424,MX,Apr 2 2025 6:12AM,"{""Sha256"":""XIjTN93lwFa1NQqkgp5Caojuh26zYulnDaT...",wamid.HBgNNTIxNDc3NzI5NTQyNBUCABIYFDVFNjVDMjBG...,50649,5215662260886
3,524433302549,MX,Apr 2 2025 8:32AM,"{""Sha256"":""LnMo81aL1UiUGjZFsN5q4vZViMWIeiyixPx...",wamid.HBgNNTIxNDQzMzMwMjU0ORUCABIYIDExRDIyNEFF...,50651,5215662260886
4,525578202314,MX,Apr 2 2025 8:39AM,"{""Sha256"":""iNhoxNVcN3sz28LLlHqcCcsQ1LhpQgKQTm6...",wamid.HBgNNTIxNTU3ODIwMjMxNBUCABIYIDIzRDM3QTI1...,50652,5215662260886


## Drop unnecessary columns

- locale
- MessageData
- ChannelType
- ChannelId

In [10]:
mx_df = mx_df.drop(['locale', 'ChannelId'], axis=1)

## Set datatypes

In [11]:
mx_df['mobile'] = mx_df['mobile'].fillna(0).astype(int)
mx_df['message_hcp_reply'] = mx_df['message_hcp_reply'].astype(str)
# mx_df['createddate'] = mx_df['createddate'].astype()
mx_df['ConversationId'] = mx_df['ConversationId'].astype(str)
mx_df['ChatMessagingMOLogId'] = mx_df['ChatMessagingMOLogId'].astype(int)


## Json Parse

In [12]:
def extracting_json_data(data: dict):
    response = ''

    #Body strcuture: Simple answer
    if 'Body' in data.keys():
        response = data['Body']
        return response

    # Payload: Button response
    if 'Payload' in data.keys():
        response = data['Payload']
        return response
    
    # Multimedia resource
    if 'Sha256' in data.keys() or 'Id' in data.keys():

        if 'Caption' in data.keys():
            response = data['Caption']

        if 'FileName' in data.keys():
            response = response +'\n'+ data['FileName'] if response != '' else data['FileName']
        
        if  'MimeType' in data.keys():       
            response = response + '\n' + 'Recurso multimedia enviado: ' + data['MimeType'] if response != '' else 'Recurso multimedia enviado: ' + data['MimeType']

        return response
    
    # Long answer
    # if 'TextMessage' in data.keys():
    #     if 'Text' in data['TextMessage'].keys() and data['TextMessage'].keys()!= None:
    #         response = data['TextMessage']['Text']
    #     return response
    
    return response



In [13]:
def parse_json(response: str) -> str:

    response =  re.sub(r'\\(?!n)', '', response)
    response_data = ''

    if response[:2] != '{"':
        return response
        
    # Cleaning Json
    if response[-1] != '}':
        response = response + '}'

    if response[-2] != '"' and response[-5:-1] != 'null':
        response = response[:-1] + '"}'

    if response[-4:] == '","}':
        response = response[:-4] + '"}'
        
    response = response.replace('}"}', '}}')
    
    try:
        json_data = json.loads(response)
        response_data = extracting_json_data(json_data)
        return response_data
    
    except json.JSONDecodeError as e:
        # print(response)
        # print(f"Todavía hay un error: {e}")
        response = re.sub(r"(?<![{:}'])\"(?![{:}'])", r'\"', response)
        # print(response)

        try:
            json_data = json.loads(response)
            response_data = extracting_json_data(json_data)
            return response_data
        
        except json.JSONDecodeError as e:
            return ''

In [14]:
# json_messages = mx_df['message_hcp_reply']

In [15]:
# for message in json_messages:
    # parse_json(message)

In [16]:
mx_df['message_hcp_reply'] = mx_df['message_hcp_reply'].apply(parse_json)

In [17]:
mx_df.head()

,mobile,createddate,message_hcp_reply,ConversationId,ChatMessagingMOLogId
0,524499287753,Apr 1 2025 8:20PM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNDQ5OTI4Nzc1MxUCABIYIDM4MTc1MEI5...,50647
1,526641650361,Apr 1 2025 10:13PM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNjY0MTY1MDM2MRUCABIYFDNBNzFFNTkx...,50648
2,524777295424,Apr 2 2025 6:12AM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNDc3NzI5NTQyNBUCABIYFDVFNjVDMjBG...,50649
3,524433302549,Apr 2 2025 8:32AM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNDQzMzMwMjU0ORUCABIYIDExRDIyNEFF...,50651
4,525578202314,Apr 2 2025 8:39AM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNTU3ODIwMjMxNBUCABIYIDIzRDM3QTI1...,50652


In [18]:
# Añadir una columna con el orden de aparición de los IDs compartidos
mx_df['message_number'] = mx_df.groupby('ConversationId').cumcount() + 1


In [19]:
mx_df.head()

,mobile,createddate,message_hcp_reply,ConversationId,ChatMessagingMOLogId,message_number
0,524499287753,Apr 1 2025 8:20PM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNDQ5OTI4Nzc1MxUCABIYIDM4MTc1MEI5...,50647,1
1,526641650361,Apr 1 2025 10:13PM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNjY0MTY1MDM2MRUCABIYFDNBNzFFNTkx...,50648,1
2,524777295424,Apr 2 2025 6:12AM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNDc3NzI5NTQyNBUCABIYFDVFNjVDMjBG...,50649,1
3,524433302549,Apr 2 2025 8:32AM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNDQzMzMwMjU0ORUCABIYIDExRDIyNEFF...,50651,1
4,525578202314,Apr 2 2025 8:39AM,Recurso multimedia enviado: image/jpeg,wamid.HBgNNTIxNTU3ODIwMjMxNBUCABIYIDIzRDM3QTI1...,50652,1


## Create new dataframes

### Historic_Messages

In [20]:
his_messages = mx_df[['ChatMessagingMOLogId', 'ConversationId', 'message_hcp_reply', 'message_number']].copy()
his_messages.rename({
    'ChatMessagingMOLogId': 'message_id', 
    'ConversationId': 'conversation_id',
    'message_hcp_reply': 'message'}, 
    axis=1, inplace=True)

In [21]:
his_messages.sort_values(by='message_id', inplace=True)

In [22]:
his_messages.head()

,message_id,conversation_id,message,message_number
6753,35924,01J0Q8BV98KWR1Q49J1FX25BQ9,,2
6754,35931,01J0RQTJSV0B7QS7X3MFWG92Z9,,1
6755,35932,01J0Q8BV98KWR1Q49J1FX25BQ9,,3
19,35939,01J0Q8BV98KWR1Q49J1FX25BQ9,,1
8008,35957,01J0Q8BV98KWR1Q49J1FX25BQ9,,4


In [23]:
# his_messages.to_csv('C:/Users/U1058105/OneDrive - Sanofi/Documentos/WA Responses Data/final data/CO_historic_messages.csv', index=False)
his_messages.to_csv('../data/processed/MX_historic_messages.csv', index=False)

### Conversations_historic
    - conversation_Id(Unico): Identificador de la sesión
    - mobile_phone: Número telefonico de la persona que manda el mensaje.
    - campaing_id: Id de la campaña a la que se respondio. Se relaciona con campaing_historic.
    - date: Fecha de la conversación
    - conversation_category: Categoría del mensaje: mensaje automatico, url o mensaje personalizado.
    - conversation_description: Descripción breve sobre la conversación.

In [ ]:
conv_his = mx_df[['ConversationId', 'mobile', 'createddate']].copy()
conv_his.rename(
    'ConversationId': 'conversation_id',
    'createddate': 'date'}, 
    axis=1, inplace=True)

In [25]:
conv_his.head()

,conversation_id,mobile,date
0,wamid.HBgNNTIxNDQ5OTI4Nzc1MxUCABIYIDM4MTc1MEI5...,524499287753,Apr 1 2025 8:20PM
1,wamid.HBgNNTIxNjY0MTY1MDM2MRUCABIYFDNBNzFFNTkx...,526641650361,Apr 1 2025 10:13PM
2,wamid.HBgNNTIxNDc3NzI5NTQyNBUCABIYFDVFNjVDMjBG...,524777295424,Apr 2 2025 6:12AM
3,wamid.HBgNNTIxNDQzMzMwMjU0ORUCABIYIDExRDIyNEFF...,524433302549,Apr 2 2025 8:32AM
4,wamid.HBgNNTIxNTU3ODIwMjMxNBUCABIYIDIzRDM3QTI1...,525578202314,Apr 2 2025 8:39AM


In [26]:
conv_his = conv_his.drop_duplicates(subset=['conversation_id'], keep='first')

In [27]:
len(conv_his)

8349

In [28]:
conv_his['campaing_id'] = ''
conv_his['conversation'] = ''
conv_his['conversation_category'] = ''
conv_his['conversation_feeling'] = ''
conv_his['conversation_need'] = ''
conv_his['support_type'] = ''

In [29]:
conv_his = conv_his[['conversation_id', 'mobile', 'date', 'campaing_id', 'conversation', 'conversation_category', 'conversation_feeling', 'conversation_need', 'support_type']]

In [30]:
conv_his['date'] = pd.to_datetime(conv_his['date'], format="mixed")

### Add conversations

In [31]:
conversations_id = conv_his['conversation_id']

In [32]:
for id in conversations_id:
    try:
        messages_text = his_messages[his_messages['conversation_id'] == id].sort_values(by='message_number')['message'].astype(str).tolist()
        if not messages_text:
            raise ValueError("messages_text is an empty list")
    except Exception as e:
        print(f"Error processing conversation_id {id}: {e}")
        messages_text = []
    final_message = '\n'.join(messages_text) if len(messages_text) > 0 else ''

    conv_his.loc[conv_his['conversation_id'] == id, 'conversation'] = final_message

In [33]:
conv_his.head()

,conversation_id,mobile,date,campaing_id,conversation,conversation_category,conversation_feeling,conversation_need,support_type
0,wamid.HBgNNTIxNDQ5OTI4Nzc1MxUCABIYIDM4MTc1MEI5...,524499287753,2025-04-01 20:20:00,,Recurso multimedia enviado: image/jpeg,,,,
1,wamid.HBgNNTIxNjY0MTY1MDM2MRUCABIYFDNBNzFFNTkx...,526641650361,2025-04-01 22:13:00,,Recurso multimedia enviado: image/jpeg,,,,
2,wamid.HBgNNTIxNDc3NzI5NTQyNBUCABIYFDVFNjVDMjBG...,524777295424,2025-04-02 06:12:00,,Recurso multimedia enviado: image/jpeg,,,,
3,wamid.HBgNNTIxNDQzMzMwMjU0ORUCABIYIDExRDIyNEFF...,524433302549,2025-04-02 08:32:00,,Recurso multimedia enviado: image/jpeg,,,,
4,wamid.HBgNNTIxNTU3ODIwMjMxNBUCABIYIDIzRDM3QTI1...,525578202314,2025-04-02 08:39:00,,Recurso multimedia enviado: image/jpeg,,,,


In [34]:
conv_his.sort_values(by='date', ascending=False, inplace=True)
conv_his.to_csv('../data/processed/MX_historic_conversations.csv', index=False)